# Creando y generando archivos de entrada .in para Quantum Expresso

## Importando liberías a utilizar

In [1]:
import os
import pandas as pd
import numpy as np

## Definiendo directorio principal

In [2]:
directorio_principal = '/home/marcocarmonaga/Documents/MoB_2-Study-Properties_Quantum_expresso/notebooks-scripts'
os.chdir(directorio_principal)

## Cambiando a carpeta sobre la que se trabajará (directorio secundario)

In [3]:
scf_path = '../data/processed/scf'
os.chdir(scf_path)

# Visualizando datos con los que se trabajan

In [4]:
df = pd.read_csv('../presion-Volumen-c_a-valores-especificos.csv')
df

,Presion_GPa,Volumenes,c_a
0,0,177.91000,1.10154
1,50,156.88202,1.08218
2,65,152.59598,1.07871
3,70,151.29232,1.07768
4,80,148.84408,1.07580
5,90,146.58297,1.07411
6,100,144.48417,1.07258
7,110,142.52742,1.07119
8,120,140.69601,1.06991
9,130,138.97594,1.06874


## Definiendo función para obtener el valor del lattice parametro $a$ a partir de la siguiente ecuación

$$
V = \frac{\sqrt{3}}{2}a^3(c/a)\quad \Rightarrow \quad a= \sqrt[3]{\frac{2V}{\sqrt{3}\cdot(c/a)}}
$$

Considerando que conocemos el valor de $V$ y $c/a$.

In [5]:
def a(V:float,c_a:float) -> float:
    return round(np.cbrt((2 * V) / (np.sqrt(3) * c_a)),5)

# Anexando columna para valores de $a$

In [6]:
df_tolist = df.loc[:,['Volumenes','c_a']].to_numpy().tolist()

In [7]:
a_values = list()
for df_tonumpy_i in df_tolist:
    a_value = a(df_tonumpy_i[0],df_tonumpy_i[1])
    a_values.append(a_value)

In [8]:
df['a'] = a_values
df

,Presion_GPa,Volumenes,c_a,a
0,0,177.91000,1.10154,5.71334
1,50,156.88202,1.08218,5.51122
2,65,152.59598,1.07871,5.46641
3,70,151.29232,1.07768,5.45254
4,80,148.84408,1.07580,5.42613
5,90,146.58297,1.07411,5.40134
6,100,144.48417,1.07258,5.37799
7,110,142.52742,1.07119,5.35592
8,120,140.69601,1.06991,5.33500
9,130,138.97594,1.06874,5.31511


# Definiendo función para generar archivos .in

In [9]:
def gen_nscf_file(a_value,c_a_value):
   crystal = r'{crystal}'
   nscf_file = f''' &CONTROL
    calculation='nscf',
    prefix='mob2',
    pseudo_dir='/home/marcocarmonaga/qe-7.0/pseudo',
    outdir='./tmp',
!    wf_collect=.true.,
 /
 &SYSTEM
    ibrav =  4,
    celldm(1) = {a_value},
    celldm(3) = {c_a_value},
    nat =  3,
    ntyp = 2,
    nbnd = 20,
    ecutwfc = 36.0,
    occupations = 'tetrahedra_opt',
 !   occupations = 'smearing',
 !   smearing = 'gauss',
 !   degauss = 0.015,
 /
 &ELECTRONS
    diagonalization = 'davidson'
    mixing_mode = 'plain',
    conv_thr =  1.0d-8
    mixing_beta = 0.7 
 /
ATOMIC_SPECIES
   Mo  95.95  Mo_ONCV_PBE-1.2.upf
   B   10.81  B_ONCV_PBE-1.2.upf
ATOMIC_POSITIONS {crystal}
   Mo 0.00 0.00 0.00
   B .6666666666666666  .3333333333333333  0.5
   B .3333333333333333  .6666666666666666  0.5
K_POINTS automatic
   36 36 36   0 0 1
!   18 18 18   0 0 1
'''
   return nscf_file

# Anexando columna de texto para archivo .in

In [10]:
c_a_a_values = df.loc[:,['c_a','a']].to_numpy().tolist()

nscf_text_list = list()
for c_a_a_value in c_a_a_values:
    nscf_text = gen_nscf_file(c_a_a_value[1],c_a_a_value[0])
    nscf_text_list.append(nscf_text)

In [11]:
df['in_file'] = nscf_text_list
df

,Presion_GPa,Volumenes,c_a,a,in_file
0,0,177.91000,1.10154,5.71334,"&CONTROL\n calculation='nscf',\n prefix..."
1,50,156.88202,1.08218,5.51122,"&CONTROL\n calculation='nscf',\n prefix..."
2,65,152.59598,1.07871,5.46641,"&CONTROL\n calculation='nscf',\n prefix..."
3,70,151.29232,1.07768,5.45254,"&CONTROL\n calculation='nscf',\n prefix..."
4,80,148.84408,1.07580,5.42613,"&CONTROL\n calculation='nscf',\n prefix..."
5,90,146.58297,1.07411,5.40134,"&CONTROL\n calculation='nscf',\n prefix..."
6,100,144.48417,1.07258,5.37799,"&CONTROL\n calculation='nscf',\n prefix..."
7,110,142.52742,1.07119,5.35592,"&CONTROL\n calculation='nscf',\n prefix..."
8,120,140.69601,1.06991,5.33500,"&CONTROL\n calculation='nscf',\n prefix..."
9,130,138.97594,1.06874,5.31511,"&CONTROL\n calculation='nscf',\n prefix..."


# Creando directorios y archivos .in

In [12]:
df_tolist = df.to_numpy().tolist()

In [13]:
df_tolist

[[0,
  177.91,
  1.10154,
  5.71334,
  " &CONTROL\n    calculation='nscf',\n    prefix='mob2',\n    pseudo_dir='/home/marcocarmonaga/qe-7.0/pseudo',\n    outdir='./tmp',\n!    wf_collect=.true.,\n /\n &SYSTEM\n    ibrav =  4,\n    celldm(1) = 5.71334,\n    celldm(3) = 1.10154,\n    nat =  3,\n    ntyp = 2,\n    nbnd = 20,\n    ecutwfc = 36.0,\n    occupations = 'tetrahedra_opt',\n !   occupations = 'smearing',\n !   smearing = 'gauss',\n !   degauss = 0.015,\n /\n &ELECTRONS\n    diagonalization = 'davidson'\n    mixing_mode = 'plain',\n    conv_thr =  1.0d-8\n    mixing_beta = 0.7 \n /\nATOMIC_SPECIES\n   Mo  95.95  Mo_ONCV_PBE-1.2.upf\n   B   10.81  B_ONCV_PBE-1.2.upf\nATOMIC_POSITIONS {crystal}\n   Mo 0.00 0.00 0.00\n   B .6666666666666666  .3333333333333333  0.5\n   B .3333333333333333  .6666666666666666  0.5\nK_POINTS automatic\n   36 36 36   0 0 1\n!   18 18 18   0 0 1\n"],
 [50,
  156.88202,
  1.08218,
  5.51122,
  " &CONTROL\n    calculation='nscf',\n    prefix='mob2',\n    pse

In [15]:
for df_tolist_i in df_tolist:
    os.chdir(str(df_tolist_i[0]))
    with open('mob2_nscf.in','w') as nscf_file:
            nscf_file.writelines(df_tolist_i[4])
            nscf_file.close()
    os.chdir('../')